# A Battle of Cities
Analyzing similarities between neighborhoods in two cities.  
This projects reuses the core of the previous exercise grouping Toronto neighborhoods and extends it so it can be used in a particular use case: an employee moving from city A to city B, who wants to know which neighborhoods in the destination city are similar to the neighborhoods in their city of origin.  

I have used in this code as city of origin Cologne (Germany), and destination: Santiago (Chile). The way the code is written, however, would allow you to change the input cities to the ones of your liking, as long as they have an entry in the Geonames website.

Find a more detailed explanation in the final report here:  
https://github.com/raulcano/Coursera_Capstone/blob/master/Final%20Report.pdf

---
# Code

In [3]:
# we install the necessary packages
!pip install geocoder
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.display import display_html
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')


Libraries imported.


## Obtain the country codes
The purpose of this step is to get the country codes as they are used in the website itself, so we can build a query that will contain the right code.  
This step will be necessary to prepare the input variables.

In [7]:
url = 'https://www.geonames.org/postal-codes/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
country_codes = soup.find('select', {"name" : "country"})

list_countries = country_codes.findAll('option')
country_codes_text = []
for l in list_countries:
    country_codes_text.append("Country: " + l.text + " | Code: " + l['value'])
print(country_codes_text[:10]) # we print the first 10 entries to see how the codes look like

['Country:  all countries | Code: ', 'Country:  Algeria | Code: DZ', 'Country:  American Samoa | Code: AS', 'Country:  Andorra | Code: AD', 'Country:  Argentina | Code: AR', 'Country:  Australia | Code: AU', 'Country:  Austria | Code: AT', 'Country:  Bangladesh | Code: BD', 'Country:  Belarus | Code: BY', 'Country:  Belgium | Code: BE']


## Prepare the input variables
The latitude and longitude will be filled automatically later on in the code. The country name will be used later to get the coordinates.

In [8]:
cities = [
    {'city' : 'Cologne', 'country_code' : 'DE', 'country' : 'Germany', 'latitude' : '', 'longitude' : ''},
    {'city' : 'Santiago', 'country_code' : 'CL', 'country' : 'Chile', 'latitude' : '', 'longitude' : ''},
]

## Create a dataframe with postal codes, coordinate and neighborhoods
The dataframe shall be a merge of all the data from all the input cities

In [13]:
dataframes = []
for city in cities:
    url = 'https://www.geonames.org/postalcode-search.html?q=' + city['city'] + '&country=' + city['country_code']
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    #tables.append(soup.find('table', class_='restable'))
    table = soup.find('table', class_='restable')
    table_df = pd.read_html(str(table),header=0)[0]
    # add city name to the table
    table_df['City'] = city['city']
    dataframes.append(table_df)

df = pd.concat(dataframes)
df = df.reset_index(drop=True)
df.head()


,Admin1,Admin2,Admin3,Admin4,City,Code,Country,Place,Unnamed: 0
0,Nordrhein-Westfalen,Regierungsbezirk Köln,"Köln, Stadt","Köln, Stadt",Cologne,50823.0,Germany,Köln,1.0
1,NaN,NaN,NaN,NaN,Cologne,NaN,NaN,50.951/6.926,NaN
2,Nordrhein-Westfalen,Regierungsbezirk Köln,"Köln, Stadt","Köln, Stadt",Cologne,50667.0,Germany,Köln,2.0
3,NaN,NaN,NaN,NaN,Cologne,NaN,NaN,50.939/6.955,NaN
4,Nordrhein-Westfalen,Regierungsbezirk Köln,"Köln, Stadt","Köln, Stadt",Cologne,50668.0,Germany,Köln,3.0


The columns we want in our dataframe are  
PostalCode, Borough, Neighborhood, Latitude, Longitude, City, Country

Therefore, we have to cleanup the data towards that goal.  
The mapping between the columns we get from the HTML and the desired dataframe is as follows:  
  * __PostalCode__ <-- Code
  * __Borough__ <-- Place
  * __Neighborhood__ <-- Place
  * __Latitude__ <-- First part of the cell including the coordinates, before the slash sign
  * __Longitude__ <-- Second part of the cell including the coordinates, after the slash sign

In [14]:
# rename 
df.rename(columns={'Code':'PostalCode','Place':'Borough'}, inplace=True)

# create a column neighborhood
df['Neighborhood'] = df['Borough']

# add columns "Latitude" and "Longitude" , initialized at 0
df['Latitude'] = 0.0 
df['Longitude'] = 0.0

df['PostalCode'].fillna(0, inplace=True)
convert_dict = {'PostalCode' : int, 'Latitude' : float, 'Longitude' : float, 'Borough' : str, 'Neighborhood' : str, 'City' : str, 'Country' : str}
df = df.astype(convert_dict) 

# change the order of the columns and retain only the necessary ones
df = df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude', 'City', 'Country']]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,City,Country
0,50823,Köln,Köln,0.0,0.0,Cologne,Germany
1,0,50.951/6.926,50.951/6.926,0.0,0.0,Cologne,nan
2,50667,Köln,Köln,0.0,0.0,Cologne,Germany
3,0,50.939/6.955,50.939/6.955,0.0,0.0,Cologne,nan
4,50668,Köln,Köln,0.0,0.0,Cologne,Germany


In [15]:
df.shape

(94, 7)

In [16]:
# more data wrangling. The lat/long is in the row following to each postal code
for i in range(0,len(df.index),2) :
    lat_lon = str(df.loc[i+1, 'Borough']).split("/")
    lat = lat_lon[0]
    lon = lat_lon[1]
    df.at[i, 'Latitude'] = lat
    df.at[i, 'Longitude'] = lon

df = df.iloc[::2].reset_index(drop=True)  # remove every second row
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,City,Country
0,50823,Köln,Köln,50.951,6.926,Cologne,Germany
1,50667,Köln,Köln,50.939,6.955,Cologne,Germany
2,50668,Köln,Köln,50.950,6.963,Cologne,Germany
3,50670,Köln,Köln,50.950,6.950,Cologne,Germany
4,50672,Köln,Köln,50.944,6.936,Cologne,Germany


## Use geopy library to get the latitude and longitude values of the cities
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>cities_explorer</em>, as shown below.

In [17]:
geolocator = Nominatim(user_agent="cities_explorer")
for city in cities:
    address = city['city'] + ', ' + city['country']
    location = geolocator.geocode(address)
    city['latitude'] = location.latitude
    city['longitude'] = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(city['city'], city['latitude'], city['longitude']))

The geograpical coordinates of Cologne are 50.938361, 6.959974.
The geograpical coordinates of Santiago are -33.4377968, -70.6504451.


In [18]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,City,Country
0,50823,Köln,Köln,50.951,6.926,Cologne,Germany
1,50667,Köln,Köln,50.939,6.955,Cologne,Germany
2,50668,Köln,Köln,50.950,6.963,Cologne,Germany
3,50670,Köln,Köln,50.950,6.950,Cologne,Germany
4,50672,Köln,Köln,50.944,6.936,Cologne,Germany


### We visualize cities neighborhoods superimposed on top

In [19]:
# rename the dataframe
neighborhoods = df
# neighborhoods = df.iloc[:3] # limit the amount of rows to test the code

# create maps using latitude and longitude values
maps = []
for city in cities:
    map = folium.Map(location=[city['latitude'], city['longitude']], zoom_start=15)
    # add markers to map
    for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)
    maps.append(map)

In [20]:
maps[0]

In [21]:
maps[1]

## Foursquare
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [22]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 2O5JLFN3320A552ZHOUNB3HBFMS4CYJWSYUPFMWSOVOKIQGY
CLIENT_SECRET:QGLMU1LV40ACXF34A052GTEM1V2D3L5W1CZDACVXZ4HGG5DC


In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods passed as arguments

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We run the function to get the venues in all neighborhoods in our dataframe

In [25]:
venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Köln
Santiago
Providencia
Las Condes
Vitacura
Lo Barnechea
Ñuñoa
Macul
La Reina
Penalolén
La Cisterna
El Bosque
La Florida
Independencia
Recoleta
Quinta Normal
Conchalí
Huechuraba
Renca
Quilicura
La Granja
La Pintana
San Miguel
Lo Prado
Pudahuel
Cerro Navia
Lo Espejo
Cerrillos
Maipu
Pedro Aguirre Cerda
San Ramón
San Joaquín
Estación Central


Let´s check the venues dataframe

In [27]:
print(venues.shape)
venues.head(10)

(923, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Köln,50.951,6.926,Café Sehnsucht,50.949970,6.923213,Café
1,Köln,50.951,6.926,Kaffeebud Ehrenfeld,50.951651,6.922660,Café
2,Köln,50.951,6.926,Van Dyck Rösterei,50.949294,6.922661,Coffee Shop
3,Köln,50.951,6.926,Pfeiler Grill,50.951589,6.925579,BBQ Joint
4,Köln,50.951,6.926,Pizza e Caffé,50.951718,6.925359,Pizza Place
5,Köln,50.951,6.926,Haus Scholzen,50.947137,6.923044,German Restaurant
6,Köln,50.951,6.926,Taverne Alekos,50.947734,6.921719,Taverna
7,Köln,50.951,6.926,Café Schwesterherz,50.946854,6.923819,Café
8,Köln,50.951,6.926,Café Lia,50.950585,6.927693,Coffee Shop
9,Köln,50.951,6.926,Maison Baguette,50.947360,6.922932,Sandwich Place


How many venues were returned per Neighborhood?

In [28]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Cerrillos,4,4,4,4,4,4
Cerro Navia,4,4,4,4,4,4
Conchalí,4,4,4,4,4,4
El Bosque,4,4,4,4,4,4
Estación Central,8,8,8,8,8,8
Independencia,10,10,10,10,10,10
Köln,667,667,667,667,667,667
La Cisterna,13,13,13,13,13,13
La Florida,1,1,1,1,1,1


Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 187 uniques categories.


## Analyze Each Neighborhood

In [30]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]]  + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,ATM,African Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,BBQ Joint,...,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Yoga Studio
0,Köln,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Köln,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Köln,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Köln,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Köln,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [31]:
onehot.shape

(923, 188)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,ATM,African Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,BBQ Joint,...,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Yoga Studio
0,Cerrillos,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
1,Cerro Navia,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,Conchalí,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,El Bosque,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,Estación Central,0.000000,0.000000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
5,Independencia,0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,Köln,0.001499,0.001499,0.001499,0.007496,0.005997,0.00,0.002999,0.000000,0.004498,...,0.001499,0.007496,0.007496,0.014993,0.005997,0.004498,0.001499,0.005997,0.0000,0.001499
7,La Cisterna,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
8,La Florida,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
9,La Granja,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


Let´s confirm the new size

In [33]:
grouped.shape

(29, 188)

Let's print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Cerrillos----
                  venue  freq
0         Grocery Store  0.25
1                 Plaza  0.25
2            Restaurant  0.25
3  Fast Food Restaurant  0.25
4                   ATM  0.00


----Cerro Navia----
           venue  freq
0  Grocery Store  0.25
1       Pharmacy  0.25
2         Bakery  0.25
3   Liquor Store  0.25
4            ATM  0.00


----Conchalí----
                venue  freq
0            Pharmacy  0.25
1  Athletics & Sports  0.25
2        Liquor Store  0.25
3                 Gym  0.25
4                 ATM  0.00


----El Bosque----
                           venue  freq
0                        Brewery  0.25
1                    Pizza Place  0.25
2                     Food Truck  0.25
3                   Liquor Store  0.25
4  Paper / Office Supplies Store  0.00


----Estación Central----
                  venue  freq
0                 Plaza  0.12
1                  Park  0.12
2          Soccer Field  0.12
3  Fast Food Restaurant  0.12
4         Grocery Store 

Let's put that into a *pandas* dataframe.  
First, let's write a function to sort the venues in descending order.

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)



In [37]:
neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cerrillos,Plaza,Fast Food Restaurant,Restaurant,Grocery Store,Yoga Studio,Eastern European Restaurant,Food,Flea Market,Farmers Market,Falafel Restaurant
1,Cerro Navia,Liquor Store,Pharmacy,Grocery Store,Bakery,Yoga Studio,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
2,Conchalí,Liquor Store,Pharmacy,Athletics & Sports,Gym,Yoga Studio,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
3,El Bosque,Brewery,Liquor Store,Pizza Place,Food Truck,Football Stadium,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
4,Estación Central,Soccer Field,Plaza,Asian Restaurant,Fast Food Restaurant,Farmers Market,Grocery Store,Park,Martial Arts Dojo,Electronics Store,Food & Drink Shop
5,Independencia,Fried Chicken Joint,BBQ Joint,Food,Big Box Store,Sandwich Place,Restaurant,Football Stadium,Diner,Asian Restaurant,Department Store
6,Köln,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
7,La Cisterna,Pizza Place,Gym,Sushi Restaurant,Pharmacy,Fast Food Restaurant,Other Great Outdoors,Bakery,Basketball Court,Bus Station,Peruvian Restaurant
8,La Florida,Gym / Fitness Center,Yoga Studio,Electronics Store,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,La Granja,Grocery Store,Salad Place,Farmers Market,Gym,Theater,Drugstore,Food,Flea Market,Fast Food Restaurant,Falafel Restaurant


## Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [38]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 0, 0, 0, 0, 0, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged = merged.dropna() # remove the rows where we got NaN after the merge

merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,50823,Köln,Köln,50.951,6.926,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
1,50667,Köln,Köln,50.939,6.955,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
2,50668,Köln,Köln,50.950,6.963,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
3,50670,Köln,Köln,50.950,6.950,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
4,50672,Köln,Köln,50.944,6.936,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant


Finally, let's visualize the resulting clusters

In [40]:
# create maps with clusters
all_maps_clusters = []

for city in cities:
    map_clusters = folium.Map(location=[city['latitude'], city['longitude']], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    
    all_maps_clusters.append(map_clusters)

We examine the map of clusters for the first city

In [41]:
all_maps_clusters[0]

Now, the map of clusters for the second city

In [42]:
all_maps_clusters[1]

## Examine Clusters


### Cluster 1: Dining out in German restaurants

In [43]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
1,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
2,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
3,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
4,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
5,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
6,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
7,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
8,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant
9,Köln,Cologne,Germany,0.0,Café,Italian Restaurant,German Restaurant,Bar,Bakery,Hotel,Plaza,Supermarket,Sushi Restaurant,Restaurant


### Cluster 2: Varied services

In [44]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Las Condes,Santiago,Chile,1.0,Mountain,Yoga Studio,Elementary School,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market


### Cluster 3 : Groceries and markets

In [45]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[2] + list(range(5, merged.shape[1]))]]

,Neighborhood,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Penalolén,Santiago,Chile,2.0,Garden Center,Bus Station,Garden,Yoga Studio,Football Stadium,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market


### Cluster 4

In [46]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[3] + list(range(5, merged.shape[1]))]]

,Latitude,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,-33.529,Santiago,Chile,3.0,Gym / Fitness Center,Yoga Studio,Electronics Store,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant


### Cluster 5

In [47]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(4, merged.shape[1]))]]

,Borough,Longitude,City,Country,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Renca,-70.728,Santiago,Chile,4.0,Plaza,Convenience Store,Gym / Fitness Center,Grocery Store,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
41,Cerrillos,-70.717,Santiago,Chile,4.0,Plaza,Fast Food Restaurant,Restaurant,Grocery Store,Yoga Studio,Eastern European Restaurant,Food,Flea Market,Farmers Market,Falafel Restaurant
